In [1]:
import os
import webbrowser
import msal
from msal import PublicClientApplication, ConfidentialClientApplication
import requests
import os
import json
import ast
import pandas as pd
import shutil
import base64
import datetime
import traceback
from collections import Counter
import matplotlib.pyplot as plt


In [2]:
%pwd

'c:\\Users\\lee_jayyang\\PythonProjects\\email_license\\test'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\lee_jayyang\\PythonProjects\\email_license'

In [4]:
# first get the credentials from config
from src.email_license.constants import *
from src.email_license.utils.common import read_yaml
from src.email_license.config.configuration import ConfigurationManager
from src.email_license import logger

In [5]:
config = ConfigurationManager()
api_config = config.get_api_config()
CLIENT_ID = api_config.client_id
CLIENT_SECRET = api_config.client_secret
TENANT_ID = api_config.tenant_id
base_url = api_config.endpoint
AUTHORITY = f'https://login.microsoftonline.com/{TENANT_ID}'
SCOPE = ['https://graph.microsoft.com/.default']



[2024-12-03 15:29:54,428: INFO: common yaml file: config\config.yaml loaded succesfully]


In [ ]:
def authenticate_graph_api(clnt_id, clnt_secret, tnt_id):
    access_token = None

    # default_scope = 'https%3A%2F%2Fgraph.microsoft.com%2F.default'
    # grant_type = 'client_credentials'

    try:
        client_instance = msal.ConfidentialClientApplication(
            client_id = clnt_id,
            client_credential = clnt_secret,
            authority = AUTHORITY,
        )

        logger.info("Create client instance..... Call MSAL library")

        token_response = client_instance.acquire_token_for_client(scopes=SCOPE)

        if 'access_token' in token_response:
            access_token = token_response['access_token']
            return access_token
        else:
            print("Error acquiring token:", token_response.get("error_description"))
    except Exception as e:
        logger.info("Error during authentication with MS GRAPH API")



In [ ]:
# Fetch audit logs from Graph API
def fetch_audit_logs(access_token):
    endpoint = f"{base_url}/auditLogs/signIns"
    headers = {"Authorization": f"Bearer {access_token}"}
    logs = []
    
    while endpoint:
        response = requests.get(endpoint, headers=headers)
        if response.status_code == 200:
            data = response.json()
            logs.extend(data.get("value", []))
            endpoint = data.get("@odata.nextLink", None)  # Handle pagination
        else:
            print(f"Error fetching audit logs: {response.status_code} {response.text}")
            break
    return logs


In [ ]:
# Save logs to a CSV file
def save_logs_to_csv(logs, output_path):
    if logs:
        df = pd.DataFrame(logs)
        df.to_csv(output_path, index=False)
        print(f"Audit logs saved to {output_path}")
    else:
        print("No logs available to save.")


In [ ]:
if __name__=="__main__":
    authenticate_graph_api(CLIENT_ID, CLIENT_SECRET)


[2024-12-03 15:30:43,775: INFO: 1725306364 Error during authentication with MS GRAPH API]
